In [0]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable

In [0]:
df = spark.sql("select * from fmcg.bronze.customers")
display(df)

In [0]:
#remove duplicates
df2 = spark.sql(
    """
    select distinct customer_id,customer_name,city,read_timestamp,_metadata_filename,_metadata_file_size from fmcg.bronze.customers;
    """
)
display(df2)
df2.createOrReplaceTempView("remove_duplicates")

In [0]:
df3 = df2.withColumn("customer_name", F.trim(df2["customer_name"]))
display(df3)
df3.createOrReplaceTempView("trim_table")

In [0]:
df4 = df3.withColumn("city", F.trim(df3["city"]))
df4 = df4.withColumn("city", F.when(df4["city"].isNull(), F.lit(None))
                            .when(df4["city"]== "Bengaluruu", "Bengaluru")
                            .when(df4["city"]== "Bengalore","Bengaluru")
                            .when(df4["city"]== "Hyderabadd", "Hyderabad")
                            .when(df4["city"]== "Hyderbad", "Hyderabad")
                            .when(df4["city"]== "NewDelhee", "New Delhi")
                            .when(df4["city"]== "NewDheli", "New Delhi")
                            .when(df4["city"]== "NewDelhi", "New Delhi")
                            .otherwise(df4["city"])
                  )
df4.createOrReplaceTempView("city_ok")
display(df4)


In [0]:
df5 = df4.withColumn("customer_name",F.initcap(F.col("customer_name")))
df5.createOrReplaceTempView("customer_name_ok")

In [0]:
# 789403 -new delhi
# 789420 bengaluru
# 789521 hyd
# 789603 ->hyd

df6 = df5.withColumn(
    "city",
    F.when(df5["customer_id"] == 789403, "New Delhi")
     .when(df5["customer_id"] == 789420, "Bengaluru")
     .when(df5["customer_id"] == 789521, "Hyderabad")
     .when(df5["customer_id"] == 789603, "Hyderabad")
     .otherwise(df5["city"])
)
df6 = df6.withColumn("customer_id", F.col("customer_id").cast("string"))
display(df6)
df6.createOrReplaceTempView("null_city_ok")


In [0]:
df6.write.format("delta")\
    .option("delta.enableChangeDataFeed", "true")\
    .mode("overwrite") \
    .saveAsTable(f"fmcg.silver.s_customers")
        

In [0]:
df7  = spark.sql ("select * from fmcg.silver.s_customers")
display (df7)

In [0]:
df7 = df7.withColumn("market",F.lit("India"))\
        .withColumn("platform",F.lit("Sports Bar"))\
        .withColumn("channel",F.lit("Acquisition"))\
        .withColumnRenamed("customer_id", "customer_code")\
        .withColumnRenamed("customer_name", "customer")

display(df7)

In [0]:
df_final = df7.select("customer_code", "customer","market","platform","channel")
display(df_final)

In [0]:
df_final.write.format("delta")\
    .option("delta.enableChangeDataFeed", "true")\
    .mode("overwrite") \
    .saveAsTable(f"fmcg.gold.sb_customers")

In [0]:
df_final_gold = spark.sql("select * from fmcg.gold.sb_customers")
display(df_final_gold)

In [0]:
target_table = DeltaTable.forName(spark, "fmcg.gold.dim_customers") 

target_table.alias("target").merge(
                    df_final_gold.alias("source"),
                    "target.customer_code = source.customer_code"  
                    ).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()